# Chemical equilibrium with fixed phase amount

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 7th, 2022</p>

Consider an aqueous solution at 60 °C obtained by mixing 1 kg of H{{_2}}O, 1 mol of NaCl and 1 mol of CO{{_2}}. What must the pressure of this aqueous solution be so that it is saturated with CO{{_2}}? In other words, what is the pressure that causes a bubble of CO{{_2}} to form (the bubble pressure)?

In this tutorial, we formulate a chemical equilibrium problem to calculate the CO{{_2}} bubble/saturation pressure of this aqueous saline solution. In this problem, we consider a chemical system with an aqueous and gaseous phases. Our equilibrium problem will consider **a constraint that imposes the amount of the gas phase to be a minuscule value** (to model the limit condition in which a bubble is formed).

Let's create our {{ChemicalSystem}} object with these phases:

In [2]:
from reaktoro import *

db = PhreeqcDatabase("pitzer.dat")

solution = AqueousPhase(speciate("H O Na Cl C"))
solution.setActivityModel(ActivityModelPitzerHMW())

gases = GaseousPhase("CO2(g) H2O(g)")
gases.setActivityModel(ActivityModelPengRobinson())

system = ChemicalSystem(db, solution, gases)

Next, we create an initial chemical state for this system representing our aqueous solution at 60 °C (with composition 1 molal NaCl and 1 molal CO{{_2}}):

In [3]:
state = ChemicalState(system)
state.temperature(60.0, "celsius")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("CO2", 1.0, "mol")

print("INITIAL STATE")
print(state)

INITIAL STATE
+-----------------+---------+------+
| Property        |   Value | Unit |
+-----------------+---------+------+
| Temperature     |  333.15 |    K |
| Pressure        |  100000 |   Pa |
| Charge:         |  -1e-16 |  mol |
| Element Amount: |         |      |
| :: H            | 111.012 |  mol |
| :: C            |       1 |  mol |
| :: O            | 57.5062 |  mol |
| :: Na           |       1 |  mol |
| :: Cl           |       1 |  mol |
| Species Amount: |         |      |
| :: H+           |   1e-16 |  mol |
| :: H2O          | 55.5062 |  mol |
| :: CO3-2        |   1e-16 |  mol |
| :: CO2          |       1 |  mol |
| :: Cl-          |       1 |  mol |
| :: HCO3-        |   1e-16 |  mol |
| :: Na+          |       1 |  mol |
| :: OH-          |   1e-16 |  mol |
| :: CO2(g)       |   1e-16 |  mol |
| :: H2O(g)       |   1e-16 |  mol |
+-----------------+---------+------+


We need now to create an {{EquilibriumSolver}} that can solve equilibrium problems with the following constrained properties:

* temperature; and
* amount of gaseous phase.

These constraint specifications are provided to the {{EquilibriumSpecs}} object below, which is then used to construct our {{EquilibriumSolver}} object:

In [5]:
specs = EquilibriumSpecs(system)
specs.temperature()
specs.phaseAmount("GaseousPhase")

solver = EquilibriumSolver(specs)

The next step is to create an object {{EquilibriumConditions}} in which we specify the values of **temperature** and **amount of gas phase** (which should be a small amount). We do this in the next code block, which also defines the lower and upper limits for pressure (to avoid negative and unrealistic large pressure values during the equilibrium calculation!):

In [6]:
conditions = EquilibriumConditions(specs)
conditions.temperature(60.0, "celsius")
conditions.phaseAmount("GaseousPhase", 1.0, "umol")  # umol = 1e-6 moles
conditions.setLowerBoundPressure(1.0, "bar")
conditions.setUpperBoundPressure(1000.0, "bar")

We now have everything needed to perform the calculation:

In [7]:
solver.solve(state, conditions)

print("FINAL STATE")
print(state)

FINAL STATE
+-----------------+--------------+------+
| Property        |        Value | Unit |
+-----------------+--------------+------+
| Temperature     |       333.15 |    K |
| Pressure        |  2.59734e+07 |   Pa |
| Charge:         | -1.42989e-17 |  mol |
| Element Amount: |              |      |
| :: H            |      111.012 |  mol |
| :: C            |            1 |  mol |
| :: O            |      57.5062 |  mol |
| :: Na           |            1 |  mol |
| :: Cl           |            1 |  mol |
| Species Amount: |              |      |
| :: H+           |   0.00127043 |  mol |
| :: H2O          |      55.5049 |  mol |
| :: CO3-2        |  6.61448e-10 |  mol |
| :: CO2          |     0.998729 |  mol |
| :: Cl-          |            1 |  mol |
| :: HCO3-        |   0.00127043 |  mol |
| :: Na+          |            1 |  mol |
| :: OH-          |  1.80938e-10 |  mol |
| :: CO2(g)       |  9.71929e-07 |  mol |
| :: H2O(g)       |  2.80706e-08 |  mol |
+-----------------+---

The CO{{_2}} saturation pressure for our aqueous solution is printed next: 

In [9]:
P = state.pressure() * 1e-5  # convert pressure from Pa to bar
print("Computed saturation pressure is", P, "bar")

Computed saturation pressure is 259.734 bar


And with that, we conclude this tutorial, which demonstrates how Reaktoro can perform chemical equilibrium calculations with given **temperature** and **phase amounts** to calculate the CO{{_2}} saturation/bubble pressure of an aqueous electrolyte solution.